# Traitement des variables categorielles : one-hot encoding



### Traiter les variables categorielles

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from helpers import tp1_2

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# Charger les données
iowa_file_path = 'data/iowa_data.csv'
home_data = pd.read_csv(iowa_file_path)
home_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

# Selectionner la cible
y = home_data.SalePrice

# Pour simplifier, nous allons supprimer les colonnes avec des valeurs manquantes
cols_with_missing = [col for col in home_data.columns if home_data[col].isnull().any()]
X = home_data.drop(cols_with_missing, axis=1)
X.drop(['SalePrice'], axis=1, inplace=True)


# Diviser les données en sous-ensembles d'entrainement et de validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)

tp1_2.step5.check(X_train, X_valid, y_train, y_valid)
# tp1_2.step5.hint()
# tp1_2.step5.solution()


  1 Looks good. Good job!                                                                                          

In [5]:

# fonction pour entrainer le modèle et evaluer la performance
def score_dataset(X_train, X_valid, y_train, y_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    return mean_absolute_error(y_valid, preds)

# Le code suivant donne une erreur car il y a des variables catégorielles 
score = score_dataset(X_train, X_valid, y_train, y_valid)

ValueError: could not convert string to float: 'RL'

### Supprimer les colonnes avec des données catégorielles

Vous commencerez par l'approche la plus simple. Utilisez la cellule de code ci-dessous pour prétraiter les données dans X_train et X_valid pour supprimer les colonnes avec des données catégorielles. Définissez les DataFrames prétraités sur drop_X_train et drop_X_valid, respectivement.

In [9]:
# Remplissez les lignes ci-dessous : supprimez les colonnes dans les données d'entraînement et de validation
drop_X_train = X_train.select_dtypes(exclude=['object']) 
drop_X_valid = X_valid.select_dtypes(exclude=['object'])
# # Le code suivant donne une erreur car il y a des variables catégorielles
#score = score_dataset(X_train, X_valid, y_train, y_valid)
# Le code suivant fonctionne après avoir supprimé les variables catégorielles
score = score_dataset(drop_X_train, drop_X_valid, y_train, y_valid)
print (score)

tp1_2.step6.check(drop_X_train, drop_X_valid, score)
# tp1_2.step6.hint()
# tp1_2.step6.solution()


23842.062627904437


  1 Expected score to be 23842.062627904717, but got 23842.062627904437                                            

### Encodage one-hot et encodage ordinal

In [16]:
# Colonnes de catégorie dans les données d'entraînement
object_cols = [col for col in X_train.columns if X_train[col].dtype == "object"]

# Obtenir le nombre d'entrées uniques dans chaque colonne avec des données catégorielles
object_nunique = list(map(lambda col: X_train[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Afficher le nombre d'entrées uniques par colonne, par ordre croissant
sorted(d.items(), key=lambda x: x[1])

[('Street', 2),
 ('Utilities', 2),
 ('CentralAir', 2),
 ('LandSlope', 3),
 ('PavedDrive', 3),
 ('LotShape', 4),
 ('LandContour', 4),
 ('ExterQual', 4),
 ('KitchenQual', 4),
 ('MSZoning', 5),
 ('LotConfig', 5),
 ('BldgType', 5),
 ('ExterCond', 5),
 ('HeatingQC', 5),
 ('Condition2', 6),
 ('RoofStyle', 6),
 ('Foundation', 6),
 ('Heating', 6),
 ('Functional', 6),
 ('SaleCondition', 6),
 ('RoofMatl', 7),
 ('HouseStyle', 8),
 ('Condition1', 9),
 ('SaleType', 9),
 ('Exterior1st', 15),
 ('Exterior2nd', 16),
 ('Neighborhood', 25)]

### Étude de la cardinalité

La sortie ci-dessus montre, pour chaque colonne avec des données catégorielles, le nombre de valeurs uniques dans la colonne. Par exemple, la colonne "Street" dans les données d'entraînement a deux valeurs uniques : "Grvl" et "Pave", correspondant respectivement à une route de gravier et à une route goudronnée.

Nous appelons le nombre d'entrées uniques d'une variable catégorielle la cardinalité de cette variable catégorielle. Par exemple, la variable 'Street' a la cardinalité 2.

Utilisez la sortie ci-dessus pour répondre aux questions ci-dessous.

In [25]:
# Remplissez la ligne ci-dessous : Combien de variables catégorielles 
# dans les données d'apprentissage ont une cardinalité supérieure à 10 ?
high_cardinality_numcols = list(map(lambda col: X_train[col].nunique(), object_cols))
print(high_cardinality_numcols)
filtered_values = [values for values in high_cardinality_numcols if values > 10]
print(filtered_values)
high_cardinality_numcols = len(filtered_values)

# Remplissez la ligne ci-dessous : Combien de colonnes sont nécessaires
# pour encoder One-Hot la variable 'Neighborhood' dans les données d'apprentissage ?
num_cols_neighborhood = X_train['Neighborhood'].nunique()
print(num_cols_neighborhood)
tp1_2.step7.check(high_cardinality_numcols, num_cols_neighborhood)
# tp1_2.step7.hint()
# tp1_2.step7.solution()

[5, 2, 4, 4, 2, 5, 3, 25, 9, 6, 5, 8, 6, 7, 15, 16, 4, 5, 6, 6, 5, 2, 4, 6, 3, 9, 6]
[25, 15, 16]
25


  1 Looks good. Good job!                                                                                          

Pour les ensembles de données volumineux comportant de nombreuses lignes, l'encodage One-Hot peut considérablement augmenter la taille de l'ensemble de données. Pour cette raison, nous n'encoderons généralement que des colonnes avec une cardinalité relativement faible. Ensuite, les colonnes à cardinalité élevée peuvent soit être supprimées de l'ensemble de données, soit utiliser un encodage ordinal.

Par exemple, considérons un ensemble de données avec 10 000 lignes et contenant une colonne catégorielle avec 100 entrées uniques.

Si cette colonne est remplacée par l'encodage one-hot correspondant, combien d'entrées sont ajoutées à l'ensemble de données ?
Si nous remplaçons plutôt la colonne par l'encodage ordinal, combien d'entrées sont ajoutées ?
Utilisez vos réponses pour remplir les lignes ci-dessous.

In [34]:
# Remplissez la ligne ci-dessous : Combien d'entrées sont ajoutées à 
# l'ensemble de données en remplaçant la colonne par un encodage unique ?
tp1_2.step8.solution()
OH_entries_added = 1e4*100 - 1e4
print(OH_entries_added)


# Remplissez la ligne ci-dessous : Combien d'entrées sont ajoutées 
# au jeu de données en remplaçant la colonne par un encodage ordinal ?
label_entries_added = 0
print(label_entries_added)

tp1_2.step8.check(OH_entries_added, label_entries_added)
# tp1_2.step8.hint()
# tp1_2.step8.solution()


  1                                                                                                                
  2 #  1e4*100 de nouvelles entrées et 1e4 anciennes entrées seront supprimées                                     
  3 OH_entries_added = 1e4*100 - 1e4                                                                               
  4 # 1e4 nouvelles entrées replacent 1e4 anciennes entrées                                                        
  5 label_entries_added = 0                                                                                        
  6                                                                                                                

990000.0
0


  1 Looks good. Good job!                                                                                          

Ensuite, vous expérimenterez l'encodage One-Hot. Mais, au lieu d'encoder toutes les variables catégorielles dans l'ensemble de données, vous le ferez que pour les colonnes avec une cardinalité inférieure à 10.

In [50]:
# Colonnes qui seront encodées en one-hot
#tp1_2.step9.solution()
low_cardinality_cols = [col for col in object_cols if X_train[col].nunique() < 10]


# Colonnes qui seront supprimées du dataset
high_cardinality_cols = list(set(object_cols)-set(low_cardinality_cols))
high_cardinality_cols.reverse()

print('Categorical columns that will be one-hot encoded:', low_cardinality_cols)
print('\nCategorical columns that will be dropped from the dataset:', high_cardinality_cols)

tp1_2.step9.check(low_cardinality_cols, high_cardinality_cols)
# tp1_2.step9.hint()
# tp1_2.step9.solution()

Categorical columns that will be one-hot encoded: ['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'ExterQual', 'ExterCond', 'Foundation', 'Heating', 'HeatingQC', 'CentralAir', 'KitchenQual', 'Functional', 'PavedDrive', 'SaleType', 'SaleCondition']

Categorical columns that will be dropped from the dataset: ['Exterior2nd', 'Neighborhood', 'Exterior1st']


  1 Looks good. Good job!                                                                                          

Utilisez la cellule de code suivante pour encoder One-Hot les données dans `X_train` et `X_valid`. Définissez les DataFrames prétraités sur `OH_X_train` et `OH_X_valid`, respectivement.
- La liste complète des colonnes catégorielles du jeu de données se trouve dans la liste Python `object_cols`.
- Vous ne devez encoder qu'une seule fois les colonnes catégorielles dans `low_cardinality_cols`. Toutes les autres colonnes catégorielles doivent être supprimées de l'ensemble de données.

In [54]:
from sklearn.preprocessing import OneHotEncoder

tp1_2.step10.solution()

# Appliquer un encodeur one-hot à chaque colonne avec des données catégorielles
OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))

# One-hot encodage supprime l'index; le remettre
OH_cols.index = X.index

# Supprimer les colonnes catégorielles (sera remplacé par l'encodage one-hot)
num_X = X.drop(object_cols, axis=1)

# Combiner les colonnes encodées one-hot et caractéristiques numériques
OH_X = pd.concat([num_X, OH_cols], axis=1)

# Assurer que toutes les colonnes sont des strings
OH_X.columns = OH_X.columns.astype(str)

OH_X_train, OH_X_valid, y_train, y_valid = train_test_split(OH_X, y, train_size=0.8, test_size=0.2,
                                                      random_state=0)
score = score_dataset(OH_X_train, OH_X_valid, y_train, y_valid)
print("MAE from Approach 3 (One-Hot Encoding):", score) 

tp1_2.step10.check(OH_X_train, OH_X_valid, y_train, y_valid, score)
# tp1_2.step10.hint()
# tp1_2.step10.solution()

   1                                                                                                               
   2 # Appliquer un encodeur one-hot à chaque colonne avec des données catégorielles                               
   3 OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)                                             
   4 OH_cols = pd.DataFrame(OH_encoder.fit_transform(X[low_cardinality_cols]))                                     
   5 # One-hot encodage supprime l'index; le remettre                                                              
   6 OH_cols.index = X.index                                                                                       
   7 # Supprimer les colonnes catégorielles (sera remplacé par l'encodage one-hot)                                 
   8 num_X = X.drop(object_cols, axis=1)                                                                           
   9 # Ajouter les colonnes encodées one-hot aux caractéristiques numériques                                       
  10 OH_X = pd.concat([num_X, OH_cols], axis=1)                                                                    
  11 # Assurer que toutes les colonnes sont des strings                                                            
  12 OH_X.columns = OH_X.columns.astype(str)                                                                       
  13 OH_X_train, OH_X_valid, y_train, y_valid = train_test_split(OH_X, y, train_size=0.8, test_size=0.2,           
  14                                                       random_state=0)                                         
  15 score = score_dataset(OH_X_train, OH_X_valid, y_train, y_valid)                                               
  16                                                                                                               

MAE from Approach 3 (One-Hot Encoding): 24532.986003760157


/home/iut45/Etudiants/o22307937/.local/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


  1 Expected score to be 24532.98600376439, but got 24532.986003760157                                             